In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


collection records of reviews

In [ ]:
records=open("/content/drive/My Drive/Project/ProcessedText.txt","r")
record_list=records.readlines()

In [ ]:
import json
from collections import defaultdict

rev=""
all_rev=[]
all_rev_txt=[]
all_rev_rating=[]
all_rev_id=[]

for r in range(len(record_list)):
  json_data=json.loads(record_list[r])
  rev+=json_data['text']
  all_rev.append(json_data)
  all_rev_txt.append(json_data['text'])
  all_rev_rating.append(json_data['property_dict'])



Pre-Processing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

sentences=[]
review_index=[]

ind=0
for review in all_rev_txt:
  temp1=review.split('.')
  temp2=[]
  #print(temp1)
  for s in temp1:
    sentence=s.lower()
    words=word_tokenize(s)
    en_stopwords = stopwords.words('english')
    processed_words=[]
    for token in words:
          if token not in en_stopwords:
              processed_words.append(token)

    tokenizer= RegexpTokenizer(r"\w+")
    word_list=tokenizer.tokenize(' '.join(processed_words))
    word_list=[w.lower() for w in word_list if len(w)!=1]
    temp2.append(word_list)

  temp3=[]
  for s in temp2:
    if(len(s)!=0):
      temp3.append(' '.join(s))
      review_index.append(ind)

  sentences+=temp3
  ind+=1



In [ ]:
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

#sentences=nltk.sent_tokenize(rev)
sentence_list=[]
for s in sentences:
  sentence=s.lower()
  words=word_tokenize(s)
  en_stopwords = stopwords.words('english')
  processed_words=[]
  for token in words:
        if token not in en_stopwords:
            processed_words.append(token)

  tokenizer= RegexpTokenizer(r"\w+")
  word_list=tokenizer.tokenize(' '.join(processed_words))
  word_list=[w.lower() for w in word_list if len(w)!=1]
  sentence_list.append(word_list)

sentences=[]
for s in sentence_list:
  if(len(s)!=0):
    sentences.append(' '.join(s))

sentence_wordfreq=[]

for i in sentence_list:
  sentence_wordfreq.append(FreqDist(i))


In [ ]:
from typing import DefaultDict
aspect_keyword=DefaultDict()

aspect_keyword["value"]=["value","price"]
aspect_keyword["room"]=["room","space"]
aspect_keyword["location"]=["location","locate"]
aspect_keyword["cleanliness"]=["clean","dirty"]
aspect_keyword["service"]=["service","manager"]

aspect_keyword

defaultdict(None,
            {'cleanliness': ['clean', 'dirty'],
             'location': ['location', 'locate'],
             'room': ['room', 'space'],
             'service': ['service', 'manager'],
             'value': ['value', 'price']})

functions needed for aspectsegmentation

In [ ]:
all_words=[]

for s in sentences:
  for w in s.split():
    if w not in all_words:
      all_words.append(w)


In [ ]:
from collections import defaultdict

def assign_aspect(wordfreq):
  sentence_aspect=[]
  count=DefaultDict(int)
  for word in wordfreq.keys():
    for aspect, keywords in aspect_keyword.items():
      if word in keywords:
        count[aspect]+=1
  if count : 
    maxi = max(count.values())
    for aspect, cnt in count.items():
      if cnt==maxi:
        sentence_aspect.append(aspect)

  return sentence_aspect


In [ ]:
import copy
from collections import defaultdict

sentences_dup=copy.deepcopy(sentences)

Aspect Segmentation

In [ ]:

from collections import defaultdict
import heapq
import copy

aspectWordMat = defaultdict(lambda: defaultdict(int))
aspectCount= DefaultDict(int)
wordCount=DefaultDict(int)

i=1000
changed=True
while changed:
  i-=1
  sentence_aspect=[]
  for word_freq in sentence_wordfreq:
    sentence_aspect.append(assign_aspect(word_freq))

  aspect_to_sentence=DefaultDict(list)
  aspect_to_sentence.clear()
  for i in range(len(sentences)):
    if len(sentence_aspect[i])==1:
      aspect_to_sentence[sentence_aspect[i][0]].append(sentences[i])

  #print(aspect_to_sentence)

  for aspect, sentences in aspect_to_sentence.items():
              for i in range(len(sentences)):
                  for word,freq in sentence_wordfreq[i].items():
                      aspectWordMat[aspect][word]+=freq
                      aspectCount[aspect]+=freq
                      wordCount[word]+=freq
                      #print(aspect,"-",word)


  top_p_words = {}

  for aspect in aspect_keyword.keys():
    top_p_words[aspect] = []
  for word in all_words:
      maxChi = 0.0 
      maxAspect = "" 
      for aspect in aspect_keyword.keys():
        C= sum(aspectCount.values())
        C1 = aspectWordMat[aspect][word]
        C2 = wordCount[word]-C1
        C3 = aspectCount[aspect]-C1 
        C4 = C-C1 
        deno = (C1+C3)*(C2+C4)*(C1+C2)*(C3+C4)
        if deno!=0:
          chisq = (C*(C1*C4 - C2*C3)*(C1*C4 - C2*C3))/deno
        else:
          chisq = 0.0

        aspectWordMat[aspect][word] = chisq
        if aspectWordMat[aspect][word] > maxChi:
          maxChi = aspectWordMat[aspect][word]
          maxAspect = aspect
      if maxAspect!="":
        top_p_words[maxAspect].append((maxChi, word))

  changed=False
  top_p_words_copy=copy.deepcopy(top_p_words)
  for aspect in aspect_keyword.keys():
    max_words=[]
    for i in range(5):
      max_word=""
      max_val=0
      max_tup=None
      for tup in top_p_words_copy[aspect]:
        if tup[0]>max_val:
          max_val=tup[0]
          max_word=tup[1]
          max_tup=tup
      max_words.append(max_word)
      try:
        top_p_words_copy[aspect].remove(max_tup)
      except:
        pass
    
    for word in max_words:
      if word not in aspect_keyword[aspect]:
        changed=True
        aspect_keyword[aspect].append(word)


    

#aspect_keyword

ABSA

preprocessing

In [ ]:
import pandas as pd

review_sentence_df_list=[]
for r in range(len(record_list)):
  record=json.loads(record_list[r])
  name=record['name']
  loc=record['loc']
  rating_dict=record['property_dict']
  rating=(rating_dict['cleanliness']+rating_dict['location']+rating_dict['rooms']+rating_dict['service']+rating_dict['value'])/5
  
  sentences=nltk.sent_tokenize(record['text'])
  sentence_list=[]
  for s in sentences:
    sentence=s.lower()
    words=word_tokenize(s)
    en_stopwords = stopwords.words('english')
    processed_words=[]
    for token in words:
          if token not in en_stopwords:
              processed_words.append(token)

    tokenizer= RegexpTokenizer(r"\w+")
    word_list=tokenizer.tokenize(' '.join(processed_words))
    word_list=[w.lower() for w in word_list if len(w)!=1]
    sentence_list.append(word_list)

  sentences=[]
  for s in sentence_list:
    if(len(s)!=0):
      sentences.append(' '.join(s))

  one_review=[]
  for s in sentences:
    one_review.append([name,loc,r,s,rating])

  review_sentence_df_list+=(one_review)

review_sentence_splited_data=pd.DataFrame(review_sentence_df_list,columns=['name','location','id','text','rating'])

In [ ]:
review_sentence_splited_data

,name,location,id,text,rating
0,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,0,decided family holiday destination saw ranking...,4.0
1,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,0,kept promise,4.0
2,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,0,plus staff extremely kind willing help time gr...,4.0
3,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,1,great customer service good restaurant service...,3.6
4,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,1,giulianuova pretty simple wow town clean beach...,3.6
...,...,...,...,...,...
8363,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,999,really enjoyed stay riad dar eliane authentica...,5.0
8364,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,999,shown room described picture website perfect g...,5.0
8365,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,999,dined every night raid really great food endle...,5.0
8366,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,999,star rating think well deserved overall experi...,5.0


Assigning Sentiment Score

In [ ]:

from textblob import TextBlob

sentences=[]
aspects=[]
hotel_name=[]
hotel_loc=[]
sentiments=[]

for i in range(len(review_sentence_df_list)):
  if len(sentence_aspect[i])!=0:
    try:
      sentences.append(sentences_dup[i])
      sentiments.append(TextBlob(sentences_dup[i]).sentiment.polarity)
      try:
        aspects.append(sentence_aspect[i][0])    
      except:
        aspects.append(sentence_aspect[i][0])    
      
      hotel_name.append(review_sentence_df_list[i][0])
      hotel_loc.append(review_sentence_df_list[i][1])
    except:
     pass


Assigning Aspects with cosine similarity

In [ ]:
import gensim
from gensim.models import Word2Vec

aspects_with_cosinesim=[]

for s in sentences:
  for aspect in aspect_keyword.keys():

    max_sim=0
    maxsim_aspect=""
    
    X_set = set(aspect_keyword[aspect]) 
    Y_set = set(s.split())
    l1 =[];
    l2 =[]
     
    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: 
          l1.append(1)
        else:
          l1.append(0) 
        if w in Y_set: 
          l2.append(1)
        else:
           l2.append(0)
    c = 0
      
   
    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    
    if(cosine>max_sim):
      maxsim_word=aspect
  
  aspects_with_cosinesim.append(maxsim_word)

In [ ]:
import pandas as pd

final_df_list_withsim=[]
for i in range(len(sentences)):
  temp=[hotel_name[i],hotel_loc[i],sentences[i],aspects_with_cosinesim[i],sentiments[i]]
  final_df_list_withsim.append(temp)

final_df_withsim=pd.DataFrame(final_df_list_withsim,columns=['hotel_name','hotel_loc','sentence','aspect','sentiment score'])
final_df_withsim

,hotel_name,hotel_loc,sentence,aspect,sentiment score
0,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,decided family holiday destination saw ranking...,location,0.000000
1,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,plus staff extremely kind willing help time gr...,service,0.211199
2,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,great customer service good restaurant service...,service,0.833333
3,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,giulianuova pretty simple wow town clean beach...,cleanliness,0.179167
4,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,saved vacation staying baltic,cleanliness,0.000000
...,...,...,...,...,...
6444,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,many thanks great stay,location,0.500000
6445,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,arrived medina marrakech worried state buildin...,value,-0.200000
6446,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,however open door riad transported calm quiet ...,service,0.295000
6447,Riad_Dar_Eliane,Marrakech_Marrakech_Tensift_El_Haouz_Region,recommend asking least one home cooked dinner ...,location,-0.300000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split( final_df_withsim['sentence'], final_df_withsim['aspect'], test_size=0.33, random_state=42)

naive bayes (with cosine sim)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6848285580084547


SVM ( with cosine sim )

In [ ]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support


nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', svm.SVC(decision_function_shape='ovo')),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.8872710192578676
(0.8960925662837678, 0.7772669090050931, 0.7977927582524141, None)


DataFrame for rankng

In [ ]:
final_df_list_withsim[:40]

In [ ]:
import copy

prevname=""
currentname=""

list_for_ranking=[]
aspect_df={}
temp=[]
aspect_record=[]
count=0

for record in (final_df_list_withsim):
  currentname=record[0]

  if currentname==prevname:
    if(currentname=='Agriturismi_Il_Castello_La_Grancia'):
      count+=1
      print(count," ",record[3]," ",temp[2][record[3]]," ",record[4])
      print(temp)

    try:
      temp[2][record[3]]+=record[4]
    except:
      temp[2][record[3]]=record[4]

    aspect_record.append(record[3])

  else:
    if len(temp)!=0:
      for key in temp[2].keys():
        try:
          
          #if(key=='service'):
           # print(temp[2][key]," ",aspect_record.count(key))

          temp[2][key]=temp[2][key]/aspect_record.count(key)
          #print(key," ",temp[2][key])
        except:
          pass
        

    aspect_record=[]
    
    try:
      temp.append(sum(temp[2].values())/len(temp[2].values()))
    except:
      pass

    if len(temp)!=0:
      list_for_ranking.append(copy.deepcopy(temp))

    
    if(currentname=='Agriturismi_Il_Castello_La_Grancia'):
      count+=1
      print(count," ",prevname," ",currentname," ",temp)

    temp.clear()
    temp=[]

    ascpect_df={}
    aspect_df['room']=0.0
    aspect_df['location']=0.0
    aspect_df['service']=0.0
    aspect_df['value']=0.0
    aspect_df['cleanliness']=0.0

    aspect_df[record[3]]=record[4]
    aspect_record.append(record[3])
    temp.append(record[0])
    temp.append(record[1])
    temp.append(aspect_df)

    #if(currentname=='Agriturismi_Il_Castello_La_Grancia'):
     # count+=1
      #print(count," ",temp)

  prevname=currentname

#list_for_ranking

In [ ]:
list_for_ranking

In [ ]:
import joblib

joblib.dump(list_for_ranking,'/content/drive/My Drive/list_for_ranking')
joblib.dump(nb,'/content/drive/My Drive/nb')

Function of handling query give ranked list of hotels

In [ ]:
def query_handling_and_ranking(query):
  import re
  from textblob import TextBlob
  import joblib

  list_for_ranking=joblib.load('/content/drive/My Drive/list_for_ranking')
  nb=joblib.load('/content/drive/My Drive/nb')

  #nb=joblib.load('/content/Drive/My Drive/svm_sim250k.sav')

  sentences_dup=re.split('; |, |and |but |with |in |\*|\n', query)
  sentences=[]
  for s in sentences_dup:
    sentences+=s.split('.')

  try:
    sentences.remove('')
  except:
    pass

  sentence_list=[]
  for s in sentences:
    sentence=s.lower()
    words=word_tokenize(s)
    en_stopwords = stopwords.words('english')
    processed_words=[]
    for token in words:
          if token not in en_stopwords:
              processed_words.append(token)

    tokenizer= RegexpTokenizer(r"\w+")
    word_list=tokenizer.tokenize(' '.join(processed_words))
    word_list=[w.lower() for w in word_list if len(w)!=1]
    sentence_list.append(word_list)

  sentences=[]
  for s in sentence_list:
    if(len(s)!=0):
      sentences.append(' '.join(s))

  query_sentences=[]
  aspects=[]
  sentiments=[]

  for i in range(len(sentences)):
      try:
        aspects.append(nb.predict(pd.DataFrame([sentences[i]])[0])[0])
        query_sentences.append(sentences[i])
        sentiments.append(TextBlob(sentences[i]).sentiment.polarity)
      except:
        pass

  query_df={}

  for i in range(len(aspects)):
    query_df[aspects[i]] = 0.0

  for i in range(len(aspects)):
    query_df[aspects[i]] += sentiments[i]

  for i in range(len(aspects)):
    query_df[aspects[i]] = query_df[aspects[i]]/aspects.count(aspects[i])


  aspects=['cleanliness','location','service','room','value']

  for key in aspects:
    if key not in query_df.keys():
      query_df[key] = 0.0

  final_list_for_ranking=[]
  for rev in list_for_ranking:
    flag=True
    for key in rev[2].keys():
      if rev[2][key] < query_df[key]:
        flag=False

    if flag==True:
      final_list_for_ranking.append(rev)

  
  final_list_for_ranking=(sorted(final_list_for_ranking,key=lambda l:l[-1],reverse=True))

  return final_list_for_ranking
  


taking query

In [ ]:
query=input()
output=query_handling_and_ranking(query)

want clean room


In [ ]:
output